In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# import nltk
# nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

for_sentiment = 'a person is a person no matter how small dr seuss i teach the smallest students with the biggest enthusiasm \
for learning my students learn in many different ways using all of our senses and multiple intelligences i use a wide range\
of techniques to help all my students succeed students in my class come from a variety of different backgrounds which makes\
for wonderful sharing of experiences and cultures including native americans our school is a caring community of successful \
learners which can be seen through collaborative student project based learning in and out of the classroom kindergarteners \
in my class love to work with hands on materials and have many different opportunities to practice a skill before it is\
mastered having the social skills to work cooperatively with friends is a crucial aspect of the kindergarten curriculum\
montana is the perfect place to learn about agriculture and nutrition my students love to role play in our pretend kitchen\
in the early childhood classroom i have had several kids ask me can we try cooking with real food i will take their idea \
and create common core cooking lessons where we learn important math and writing concepts while cooking delicious healthy \
food for snack time my students will have a grounded appreciation for the work that went into making the food and knowledge \
of where the ingredients came from as well as how it is healthy for their bodies this project would expand our learning of \
nutrition and agricultural cooking recipes by having us peel our own apples to make homemade applesauce make our own bread \
and mix up healthy plants from our classroom garden in the spring we will also create our own cookbooks to be printed and \
shared with families students will gain math and literature skills as well as a life long enjoyment for healthy cooking \
nannan'
ss = sid.polarity_scores(for_sentiment)

for k in ss:
    print('{0}: {1}, '.format(k, ss[k]), end='')

# we can use these 4 things as features/attributes (neg, neu, pos, compound)
# neg: 0.0, neu: 0.753, pos: 0.247, compound: 0.93

In [ ]:
def sentiment_analysis(text):
    ss = sid.polarity_scores(text)
    for k in ss:
        print('{0}: {1}, '.format(k, ss[k]), end='')


In [ ]:
sentiment_analysis('fuck you')

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# import nltk
# nltk.download('vader_lexicon')

sid = SentimentIntensityAnalyzer()

for_sentiment = 'a person is a person no matter how small dr seuss i teach the smallest students with the biggest enthusiasm \
for learning my students learn in many different ways using all of our senses and multiple intelligences i use a wide range\
of techniques to help all my students succeed students in my class come from a variety of different backgrounds which makes\
for wonderful sharing of experiences and cultures including native americans our school is a caring community of successful \
learners which can be seen through collaborative student project based learning in and out of the classroom kindergarteners \
in my class love to work with hands on materials and have many different opportunities to practice a skill before it is\
mastered having the social skills to work cooperatively with friends is a crucial aspect of the kindergarten curriculum\
montana is the perfect place to learn about agriculture and nutrition my students love to role play in our pretend kitchen\
in the early childhood classroom i have had several kids ask me can we try cooking with real food i will take their idea \
and create common core cooking lessons where we learn important math and writing concepts while cooking delicious healthy \
food for snack time my students will have a grounded appreciation for the work that went into making the food and knowledge \
of where the ingredients came from as well as how it is healthy for their bodies this project would expand our learning of \
nutrition and agricultural cooking recipes by having us peel our own apples to make homemade applesauce make our own bread \
and mix up healthy plants from our classroom garden in the spring we will also create our own cookbooks to be printed and \
shared with families students will gain math and literature skills as well as a life long enjoyment for healthy cooking \
nannan'
ss = sid.polarity_scores(for_sentiment)

for k in ss:
    print('{0}: {1}, '.format(k, ss[k]), end='')

# we can use these 4 things as features/attributes (neg, neu, pos, compound)
# neg: 0.0, neu: 0.753, pos: 0.247, compound: 0.93

## Decision Tree

### Loading Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import KeyedVectors, Word2Vec
import pickle
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score

from tqdm import tqdm_notebook as tqdm
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from scipy.sparse import hstack
from wordcloud import WordCloud
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [ ]:
df = pd.read_csv('data/preprocessed_df.csv')
print(df.shape)
df.head()

In [ ]:
y = df['project_is_approved'].values
X = df.drop(['project_is_approved'], axis=1)
X.head()

### Splitting data into Train and cross validation(or test): Stratified Sampling

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train)
print(X_train.shape, y_train.shape), print(X_cv.shape, y_cv.shape), print(X_test.shape, y_test.shape)


## Make Data Model Ready: encoding eassay, and project_title

In [ ]:
tfidf_all_features = []
tfidf_w2v_all_features = []

#### Essay

In [ ]:
vectorizer_tfidf = TfidfVectorizer(min_df=10, ngram_range=(1,4), max_features=20000)

X_train_essay_tfidf = vectorizer_tfidf.fit_transform(X_train['essay'].values)
X_cv_essay_tfidf = vectorizer_tfidf.transform(X_cv['essay'].values)
X_test_essay_tfidf = vectorizer_tfidf.transform(X_test['essay'].values)

print(X_train_essay_tfidf.shape, y_train.shape)
print(X_cv_essay_tfidf.shape, y_cv.shape)
print(X_test_essay_tfidf.shape, y_test.shape)
print("="*100)

tfidf_all_features.extend(vectorizer_tfidf.get_feature_names())


In [ ]:
list_of_sentence_train=[]
for sentence in X_train['essay']:
    list_of_sentence_train.append(sentence.split())

w2v_model = Word2Vec(list_of_sentence_train, min_count=10, size=50, workers=4)
w2v_words = list(w2v_model.wv.vocab)
tf_idf_vect = TfidfVectorizer(min_df=10, ngram_range=(1,4), max_features=20000)
tf_idf_matrix = tf_idf_vect.fit_transform(X_train['essay'])

tfidf_feat = tf_idf_vect.get_feature_names() 
dictionary = dict(zip(tf_idf_vect.get_feature_names(), list(tf_idf_vect.idf_)))

tfidf_w2v_train = []
row = 0
for sentence in tqdm(list_of_sentence_train): 
    sentence_vec = np.zeros(50)
    weight_sum = 0
    for word in sentence:
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
            tf_idf = dictionary[word] * (sentence.count(word) / len(sentence))
            sentence_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sentence_vec /= weight_sum
    tfidf_w2v_train.append(sentence_vec)
    row += 1

    
list_of_sentence_cv = []
for sentence in X_cv['essay']:
    list_of_sentence_cv.append(sentence.split())

tfidf_w2v_cv = []
row = 0
for sentence in tqdm(list_of_sentence_cv): 
    sentence_vec = np.zeros(50)
    weight_sum = 0
    for word in sentence:
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
            tf_idf = dictionary[word] * (sentence.count(word) / len(sentence))
            sentence_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sentence_vec /= weight_sum
    tfidf_w2v_cv.append(sentence_vec)
    row += 1
    
list_of_sentence_test = []
for sentence in X_test['essay']:
    list_of_sentence_test.append(sentence.split())

tfidf_w2v_test = []
row = 0
for sentence in tqdm(list_of_sentence_test): 
    sentence_vec = np.zeros(50)
    weight_sum = 0
    for word in sentence:
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
            tf_idf = dictionary[word] * (sentence.count(word) / len(sentence))
            sentence_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sentence_vec /= weight_sum
    tfidf_w2v_test.append(sentence_vec)
    row += 1
    

            

In [ ]:
tfidf_w2v = np.array([tfidf_w2v_train, tfidf_w2v_cv, tfidf_w2v_test])
tfidf = np.array([X_train_essay_tfidf, X_cv_essay_tfidf, X_test_essay_tfidf])


In [ ]:
with open('tmp/vectorizer_tfidf', 'wb') as f:
    pickle.dump(vectorizer_tfidf.get_feature_names(), f)

In [ ]:
tfidf_w2v_all_features.extend(w2v_words)

### Title

In [ ]:
vectorizer_tfidf_item = TfidfVectorizer(min_df=10, ngram_range=(1,4), max_features=5000)

X_train_title_tfidf = vectorizer_tfidf_item.fit_transform(X_train['project_title'].values.astype('U'))
X_cv_title_tfidf = vectorizer_tfidf_item.transform(X_cv['project_title'].values.astype('U'))
X_test_title_tfidf = vectorizer_tfidf_item.transform(X_test['project_title'].values.astype('U'))

print(X_train_title_tfidf.shape, y_train.shape)
print(X_cv_title_tfidf.shape, y_cv.shape)
print(X_test_title_tfidf.shape, y_test.shape)
print("="*100)

tfidf_all_features.extend(vectorizer_tfidf_item.get_feature_names())


In [ ]:
X_train['project_title'].values.astype('U')

In [ ]:
list_of_title_train=[]
for sentence in X_train['project_title'].values.astype('U'):
    list_of_title_train.append(sentence.split())

w2v_model = Word2Vec(list_of_title_train, min_count=10, size=50, workers=4)
w2v_words = list(w2v_model.wv.vocab)
tf_idf_vect = TfidfVectorizer(min_df=10, ngram_range=(1,4), max_features=20000)
tf_idf_matrix = tf_idf_vect.fit_transform(X_train['project_title'].values.astype('U'))

tfidf_feat = tf_idf_vect.get_feature_names() 
dictionary = dict(zip(tf_idf_vect.get_feature_names(), list(tf_idf_vect.idf_)))

tfidf_w2v_title_train = []
row = 0
for sentence in tqdm(list_of_title_train): 
    sentence_vec = np.zeros(50)
    weight_sum = 0
    for word in sentence:
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
            tf_idf = dictionary[word] * (sentence.count(word) / len(sentence))
            sentence_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sentence_vec /= weight_sum
    tfidf_w2v_title_train.append(sentence_vec)
    row += 1

    
list_of_title_cv = []
for sentence in X_cv['project_title'].values.astype('U'):
    list_of_title_cv.append(sentence.split())

tfidf_w2v_title_cv = []
row = 0
for sentence in tqdm(list_of_title_cv): 
    sentence_vec = np.zeros(50)
    weight_sum = 0
    for word in sentence:
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
            tf_idf = dictionary[word] * (sentence.count(word) / len(sentence))
            sentence_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sentence_vec /= weight_sum
    tfidf_w2v_title_cv.append(sentence_vec)
    row += 1
    
list_of_title_test = []
for sentence in X_test['project_title'].values.astype('U'):
    list_of_title_test.append(sentence.split())

tfidf_w2v_title_test = []
row = 0
for sentence in tqdm(list_of_title_test): 
    sentence_vec = np.zeros(50)
    weight_sum = 0
    for word in sentence:
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
            tf_idf = dictionary[word] * (sentence.count(word) / len(sentence))
            sentence_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sentence_vec /= weight_sum
    tfidf_w2v_title_test.append(sentence_vec)
    row += 1
    

            

In [ ]:
tfidf_w2v_all_features.extend(w2v_words)

In [ ]:
title_tfidf_w2v = np.array([tfidf_w2v_title_train, tfidf_w2v_title_cv, tfidf_w2v_title_test])
title_tfidf = np.array([X_train_title_tfidf, X_cv_title_tfidf, X_test_title_tfidf])


In [ ]:
# with open('tmp/vectorizer_tfidf_item', 'wb') as f:
#     pickle.dump(vectorizer_tfidf_item.get_feature_names(), f)

### Encoding categorical variable school state

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train['school_state'].values)

X_train_state_ohe = vectorizer.transform(X_train['school_state'].values)
X_cv_state_ohe = vectorizer.transform(X_cv['school_state'].values)
X_test_state_ohe = vectorizer.transform(X_test['school_state'].values)

print(X_train_state_ohe.shape, y_train.shape)
print(X_cv_state_ohe.shape, y_cv.shape)
print(X_test_state_ohe.shape, y_test.shape)
print(vectorizer.get_feature_names())
print("="*100)

tfidf_all_features.extend(vectorizer.get_feature_names())
tfidf_w2v_all_features.extend(vectorizer.get_feature_names())

### encoding categorical features: teacher_prefix

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train['teacher_prefix'].values)

X_train_teacher_ohe = vectorizer.transform(X_train['teacher_prefix'].values)
X_cv_teacher_ohe = vectorizer.transform(X_cv['teacher_prefix'].values)
X_test_teacher_ohe = vectorizer.transform(X_test['teacher_prefix'].values)

print(X_train_teacher_ohe.shape, y_train.shape)
print(X_cv_teacher_ohe.shape, y_cv.shape)
print(X_test_teacher_ohe.shape, y_test.shape)
print(vectorizer.get_feature_names())
print("="*100)

tfidf_all_features.extend(vectorizer.get_feature_names())
tfidf_w2v_all_features.extend(vectorizer.get_feature_names())

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train['project_grade_category'].values)

X_train_project_ohe = vectorizer.transform(X_train['project_grade_category'].values)
X_cv_project_ohe = vectorizer.transform(X_cv['project_grade_category'].values)
X_test_project_ohe = vectorizer.transform(X_test['project_grade_category'].values)

print(X_train_project_ohe.shape, y_train.shape)
print(X_cv_project_ohe.shape, y_cv.shape)
print(X_test_project_ohe.shape, y_test.shape)
print(vectorizer.get_feature_names())
print("="*100)

tfidf_all_features.extend(vectorizer.get_feature_names())
tfidf_w2v_all_features.extend(vectorizer.get_feature_names())

### Normalize price

In [ ]:
normalizer = Normalizer()

normalizer.fit(X_train['price'].values.reshape(-1, 1))

X_train_price_norm = normalizer.transform(X_train['price'].values.reshape(-1, 1))
X_cv_price_norm = normalizer.transform(X_cv['price'].values.reshape(-1, 1))
X_test_price_norm = normalizer.transform(X_test['price'].values.reshape(-1, 1))

print(X_train_price_norm.shape, y_train.shape)
print(X_cv_price_norm.shape, y_cv.shape)
print(X_test_price_norm.shape, y_test.shape)
print("="*100)

tfidf_all_features.extend(vectorizer.get_feature_names())
tfidf_w2v_all_features.extend(vectorizer.get_feature_names())

In [ ]:
normalizer = Normalizer()

normalizer.fit(X_train['teacher_number_of_previously_posted_projects'].values.reshape(1, -1))

X_train_prev_proj_norm = normalizer.transform(X_train['teacher_number_of_previously_posted_projects'].values.reshape(-1, 1))
X_cv_prev_proj_norm = normalizer.transform(X_cv['teacher_number_of_previously_posted_projects'].values.reshape(-1, 1))
X_test_prev_proj_norm = normalizer.transform(X_test['teacher_number_of_previously_posted_projects'].values.reshape(-1, 1))


print(X_train_prev_proj_norm.shape, y_train.shape)
print(X_cv_prev_proj_norm.shape, y_cv.shape)
print(X_test_prev_proj_norm.shape, y_test.shape)
print("="*100)

tfidf_all_features.extend(vectorizer.get_feature_names())
tfidf_w2v_all_features.extend(vectorizer.get_feature_names())

## Preparing data for TFIDF

In [ ]:
X_train_tfidf = hstack((X_train_essay_tfidf, X_train_title_tfidf, X_train_state_ohe, X_train_teacher_ohe, X_train_project_ohe, X_train_price_norm, X_train_prev_proj_norm)).tocsr()
X_cv_tfidf = hstack((X_cv_essay_tfidf, X_cv_title_tfidf, X_cv_state_ohe, X_cv_teacher_ohe, X_cv_project_ohe, X_cv_price_norm, X_cv_prev_proj_norm)).tocsr()
X_test_tfidf = hstack((X_test_essay_tfidf, X_test_title_tfidf, X_test_state_ohe, X_test_teacher_ohe, X_test_project_ohe, X_test_price_norm, X_test_prev_proj_norm)).tocsr()

print("Final Data matrix")
print(X_train_tfidf.shape, y_train.shape)
print(X_cv_tfidf.shape, y_cv.shape)
print(X_test_tfidf.shape, y_test.shape)
print("="*100)

## Preparing data for TFIDF-W2V

In [ ]:
X_train_tfidf_w2v = hstack((tfidf_w2v_train, tfidf_w2v_title_train, X_train_state_ohe, X_train_teacher_ohe, X_train_project_ohe, X_train_price_norm, X_train_prev_proj_norm)).tocsr()
X_cv_tfidf_w2v = hstack((tfidf_w2v_cv, tfidf_w2v_title_cv, X_cv_state_ohe, X_cv_teacher_ohe, X_cv_project_ohe, X_cv_price_norm, X_cv_prev_proj_norm)).tocsr()
X_test_tfidf_w2v = hstack((tfidf_w2v_test, tfidf_w2v_title_test, X_test_state_ohe, X_test_teacher_ohe, X_test_project_ohe, X_test_price_norm, X_test_prev_proj_norm)).tocsr()

print("Final Data matrix")
print(X_train_tfidf_w2v.shape, y_train.shape)
print(X_cv_tfidf_w2v.shape, y_cv.shape)
print(X_test_tfidf_w2v.shape, y_test.shape)
print("="*100)

## Applying Decision tree on processed Data

In [ ]:
def cross_validate_model(X_train, Y_train):
    parameters = { 'max_depth': [1, 5] }
    model = DecisionTreeClassifier()
    clf = GridSearchCV(model, parameters, cv=3, scoring=make_scorer(roc_auc_score), return_train_score=True, verbose=1, n_jobs=4)
    clf.fit(X_train, y_train)
    
    train_auc_mean = clf.cv_results_['mean_train_score']
    train_auc_std = clf.cv_results_['std_train_score']
    cv_auc_mean = clf.cv_results_['mean_test_score'] 
    cv_auc_std = clf.cv_results_['std_test_score']
    optimal_alpha = clf.best_params_['max_depth']
    return parameters['max_depth'], optimal_alpha, train_auc_mean, train_auc_std, cv_auc_mean, cv_auc_std

def plot_cross_validate(alphas, train_auc_mean, train_auc_std, cv_auc_mean, cv_auc_std):
    # this code is refered from: https://stackoverflow.com/a/48803361/4084039
    plt.figure(figsize=(15, 12))
    plt.plot(alphas, train_auc_mean, label='Train AUC')
    plt.semilogx(alphas, train_auc_mean, label='Mean Train score', color='blue')
    plt.gca().fill_between(alphas, train_auc_mean - train_auc_std, train_auc_mean + train_auc_std, alpha=0.3, color='blue')

    
    plt.plot(alphas, cv_auc_mean, label='CV AUC')
    plt.semilogx(alphas, cv_auc_mean, label='Mean CV score', color='red')
    plt.gca().fill_between(alphas, cv_auc_mean - cv_auc_std, cv_auc_mean + cv_auc_std, alpha=0.3, color='red')

    plt.scatter(alphas, train_auc_mean, label='Train AUC points')
    plt.scatter(alphas, cv_auc_mean, label='CV AUC points')


    plt.legend()
    plt.xlabel("alpha: hyperparameter")
    plt.ylabel("AUC")
    plt.title("alpha: hyperparameter v/s AUC")
    plt.grid(color='black', linestyle='-', linewidth=0.5)
    plt.show()
    
    
def model_auc_roc_score(X_train, y_train, X_test, y_test, optimal_alpha):
    model = DecisionTreeClassifier(max_depth=optimal_alpha)
    model.fit(X_train, y_train)

    y_train_proba = model.predict_proba(X_train)[:,1]
    y_test_proba = model.predict_proba(X_test)[:, 1]
    train_fpr, train_tpr, train_thresholds = roc_curve(y_train, y_train_proba)
    test_fpr, test_tpr, test_thresholds = roc_curve(y_test, y_test_proba)
    return model, y_train_proba, train_fpr, train_tpr, train_thresholds, y_test_proba, test_fpr, test_tpr, test_thresholds

def plot_auc_roc_score(train_fpr, train_tpr, test_fpr, test_tpr):
    plt.figure(figsize=(15, 12))
    
    plt.plot(train_fpr, train_tpr, label=f"Train AUC = {auc(train_fpr, train_tpr)}")
    plt.plot(test_fpr, test_tpr, label=f"Test AUC = {auc(test_fpr, test_tpr)}")
    plt.plot([0, 1], [0, 1], 'g--')
    plt.legend()
    plt.xlabel("False Positive Rate(FPR)")
    plt.ylabel("True Positive Rate(TPR)")
    plt.title("AUC")
    plt.grid(color='black', linestyle='-', linewidth=0.5)
    plt.show()
    
def predict(proba, thresholds, tpr, fpr):
    pred = []
    best_thres = thresholds[np.argmax(tpr * (1 - fpr))]
    for prob in proba:
        if prob >= best_thres:
            pred.append(1)
        else:
            pred.append(0)
    return best_thres, pred
    
    
def plot_confusion_matrix(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    df = pd.DataFrame(cm)
    plt.figure(figsize=(10, 7))
    sns.heatmap(df, annot=True, fmt='d')
    plt.title("Confusion Matrix for test data")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.show()
    
def get_top_and_worst_features(model, all_features, num_of_features=20):
    pos_indx = model.feature_importances_.argsort()[::-1][:num_of_features]
    neg_indx = model.feature_importances_.argsort()[:num_of_features]
    top_features = []
    worst_features = []
    for i in list(pos_indx[:num_of_features]):
        top_features.append(all_features[i])

    for i in list(neg_indx[:num_of_features]):
        worst_features.append(all_features[i])
        
    return top_features, worst_features


def build_word_cloud(features):
    features = ' '.join(features)
    wordcloud = WordCloud().generate(features)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()
    
    
def build_model(X_train, y_train, X_test, y_test, all_features, num_of_features=20):
    alphas, optimal_alpha, train_auc_mean, train_auc_std, cv_auc_mean, cv_auc_std = cross_validate_model(X_train, y_train)
    plot_cross_validate(alphas, train_auc_mean, train_auc_std, cv_auc_mean, cv_auc_std)
    model, y_train_proba, train_fpr, train_tpr, train_thresholds, y_test_proba, test_fpr, test_tpr, test_thresholds = model_auc_roc_score(X_train, y_train, X_test, y_test, optimal_alpha)
#     pdb.set_trace();
    plot_auc_roc_score(train_fpr, train_tpr, test_fpr, test_tpr)
    thresh, y_pred = predict(y_test_proba, test_thresholds, test_tpr, test_fpr)
    print(f'The best threshold value:: {thresh}')
    plot_confusion_matrix(y_test, y_pred)
    top_features, worst_features = get_top_and_worst_features(model, all_features, num_of_features)
    build_word_cloud(top_features)
    build_word_cloud(worst_features)
    return optimal_alpha, train_fpr, train_tpr, test_fpr, test_tpr
    
    


In [ ]:
import pdb

In [ ]:
optimal_alpha_tfidf, train_fpr_tfidf, train_tpr_tfidf, test_fpr_tfidf, test_tpr_tfidf = build_model(X_train_tfidf, y_train, X_test_tfidf, y_test, tfidf_all_features, num_of_features=20)


In [ ]:
from prettytable import PrettyTable
    
x = PrettyTable()

x.field_names = ["Vectorizer", "Model", "Hyperparameter: Alpha", "Train AUC", "Test AUC"]
# auc_train_bow = auc(train_fpr_bow, train_tpr_bow)
# auc_test_bow = auc(test_fpr_bow, test_tpr_bow)

auc_train_tfidf = auc(train_fpr_tfidf, train_tpr_tfidf)
auc_test_tfidf = auc(test_fpr_tfidf, test_tpr_tfidf)


# x.add_row(["BOW", "Multinomial Naive Bayes", optimal_alpha_bow, round(auc_train_bow, 2),round(auc_test_bow, 2)])
x.add_row(["TF-IDF", "Decision Tree", optimal_alpha_tfidf, round(auc_train_tfidf, 2),round(auc_test_tfidf, 2)])

print(x)